In [1]:
import os,sys
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from sklearn.externals import joblib
import matplotlib.pyplot as plt
sys.path.append('../LIB/')
from env import ENV
from sklearn.preprocessing import normalize
from tqdm import tqdm

import os
from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU,CuDNNGRU,Flatten,BatchNormalization
from keras.callbacks import Callback
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.preprocessing import text, sequence
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
import pickle
from sklearn.preprocessing.data import QuantileTransformer
from sklearn.utils import shuffle
import gc
from sklearn.metrics import log_loss

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
X_train = pd.read_pickle(ENV.lightgbm_train_764_nn.value)
print(X_train.shape)
X_test = pd.read_pickle(ENV.lightgbm_test_764_nn.value)
print(X_test.shape)
X = pd.concat([X_train.drop('TARGET',axis=1),X_test])
print(X.shape)

(307511, 764)
(48744, 763)
(356255, 763)


In [23]:
def create_main_doc(X,categorical_col):
    #1. create document
    X_cal_values = X[categorical_col].values
    X_doc = []
    for each_line in X_cal_values:
        doc = ' '.join(each_line)
        X_doc.append(doc)
        
    #2. get max feature! max unique words
    max_features = 0
    for each in tqdm(categorical_col):
        max_features+=X[each].nunique()
    print('number of unique category is : {}'.format(max_features))
    
    #3. word to sequence
    tok=text.Tokenizer(num_words=max_features,lower=True,filters='')
    tok.fit_on_texts(X_doc)
    maxlen = len(categorical_col)
    X_cal=tok.texts_to_sequences(X_doc)
    X_cal = sequence.pad_sequences(X_cal,maxlen=maxlen)
    
    return X_cal,max_features

def one_hot_encoding(df,col,pre_fix,drop=True):
    df = df.copy()
    df[col] = df[col].fillna('NA_NOT_FOUND')
    col_name_list = []
    print('before encoding, shape is: {}'.format(df.shape))
    for each in df[col].unique():
        name = str(each)
        col_name = pre_fix + '_'+ name.replace(' ','_')
        col_name_list.append(col_name)
        df[col_name] = 0
        df.loc[df[col]==each,col_name] = 1
    if drop:
        df = df.drop([col],axis=1)
    print('after encoding, shape is: {}'.format(df.shape))
    return df,col_name_list

In [4]:
categorical_col = []
for col in X.columns:
    if X[col].dtypes == object:
        categorical_col.append(col)
print('length of categorical col is: {}'.format(len(categorical_col)))

numerical_col = list(set(X.columns) - set(categorical_col))
numerical_col.remove('SK_ID_CURR')

length of categorical col is: 37


# Scale 

In [5]:
qt = QuantileTransformer(n_quantiles=10000,output_distribution='normal')
for col in tqdm(numerical_col):
    X[col] = qt.fit_transform(X[col].values.reshape(-1,1))

100%|██████████| 725/725 [00:58<00:00, 12.47it/s]


In [6]:
X_num = X[numerical_col].values

# Process categorical column

In [7]:
X_cal,num_words_main = create_main_doc(X,categorical_col)

100%|██████████| 37/37 [00:00<00:00, 52.11it/s]


number of unique category is : 463


# Create one hot columns

In [94]:
X_one_hot = X[categorical_col].copy()
for col in tqdm(categorical_col):
    X_one_hot,new_col = one_hot_encoding(X_one_hot,col,col)
X_one_hot = X_one_hot.values    

  0%|          | 0/37 [00:00<?, ?it/s]

before encoding, shape is: (356255, 37)


  3%|▎         | 1/37 [00:03<01:48,  3.02s/it]

after encoding, shape is: (356255, 94)
before encoding, shape is: (356255, 94)


  5%|▌         | 2/37 [00:03<01:07,  1.93s/it]

...

after encoding, shape is: (356255, 453)
before encoding, shape is: (356255, 453)


 97%|█████████▋| 36/37 [01:27<00:02,  2.42s/it]

after encoding, shape is: (356255, 458)
before encoding, shape is: (356255, 458)


100%|██████████| 37/37 [01:29<00:00,  2.41s/it]

after encoding, shape is: (356255, 463)


In [98]:
X_one_hot = qt.fit_transform(X_one_hot)

In [99]:
x_num_train = X_num[:307511]
x_num_test = X_num[307511:]
x_cal_train = X_cal[:307511]
x_cal_test = X_cal[307511:]
x_onehot_train = X_one_hot[:307511]
x_onehot_test = X_one_hot[307511:]
y_train = X_train.TARGET.values

In [50]:
dim_main_numeric = x_num_train.shape[1]
dim_main_cal = x_cal_train.shape[1]
dim_main_onehot = x_onehot_train.shape[1]
main_ebd_size = 300

# Create NN

In [10]:
# model = get_NN_model()
# model.summary()

In [118]:
def get_NN_model():
    main_numeric_input = Input(shape=(dim_main_numeric, ),name='main_numerical')
    main_cal_sequence = Input(shape=(dim_main_cal, ),name = 'main_categorical')
    main_onehot_input = Input(shape=(dim_main_onehot, ),name = 'main_onehot')
    x_main_cat = Embedding(num_words_main, main_ebd_size, trainable = True)(main_cal_sequence)
    x_main_cat = SpatialDropout1D(0.5)(x_main_cat)
    x_main_cat = Flatten()(x_main_cat)
    x_main_nu = BatchNormalization()(main_numeric_input)
    x_main_nu = Dense(192,activation='relu')(x_main_nu)
    x_main_nu = Dropout(0.5)(x_main_nu)
    x_main_oh = Dense(512,activation='relu')(main_onehot_input)
    x_main_oh = Dropout(0.5)(x_main_oh)
    x = concatenate([x_main_nu, x_main_cat,x_main_oh])
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.5)(x)
    preds = Dense(1, activation="sigmoid")(x)
    model = Model([main_numeric_input,main_cal_sequence,main_onehot_input], preds)
    model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])
    
    return model

class_ratio = 1
def train_each_epoch(model,**args):
    #######################################
    x_num = args['main_numerical']
    x_cat = args['main_categorical']
    y = args['y']
    batch_size = args['batch_size']
    
    x_onehot = args['main_onehot']
    ########################################
    x_num,x_cat,y = shuffle(x_num,x_cat,y)
    model.fit({'main_numerical': x_num, 'main_categorical': x_cat,'main_onehot':x_onehot}, 
              y, 
              batch_size=batch_size, 
              epochs=1,
              verbose=1,
              class_weight={0:1,1:class_ratio})
    return model

def load_model(model,filepath):
    model.load_weights(filepath)
    return model

def save_model(model, filepath):
    model.save_weights(filepath)
    
def train_each_fold(model,**args):
    
    x_num = args['main_n_train']
    x_num_val = args['main_n_val']
    x_cat = args['main_c_train']
    x_cat_val = args['main_c_val']
    y = args['y']
    y_val = args['y_val']
    filepath = args['filepath']
    reportpath = args['reportpath']
    predspath = args['predspath']
    batch_size = args.get('batch_size',512)
    total_epoch = args.get('total_epoch',400)
    patience = args.get('patience',5)
    saving = args.get('saving',True)
    
    x_onehot = args['main_oh_train']
    x_onehot_val = args['main_oh_val']
    
    
    ROC_AUC_SCORE = []
    for epoch in range(total_epoch):  
        model = train_each_epoch(model,
                                 main_numerical=x_num,
                                 main_categorical=x_cat,
                                 y=y,
                                 main_onehot = x_onehot,
                                 batch_size=batch_size)
        y_pred = model.predict({'main_numerical':x_num_val,
                                'main_categorical':x_cat_val,
                                'main_onehot':x_onehot_val},
                               batch_size=3000,
                               verbose=1)
        score = roc_auc_score(y_val,y_pred)
        logloss = log_loss(y_val,y_pred)
        if len(ROC_AUC_SCORE) == 0:
            if saving:
                save_model(model,filepath)
            best_score = 0 
            if saving:
                print('saving preds...')
                pickle.dump(y_pred,open(predspath,'wb'))
        else:
            best_score = max(ROC_AUC_SCORE)
            if score >= best_score:
                if saving:
                    print('saving model to... {}'.format(filepath))
                    save_model(model,filepath)
                    print('saving preds...')
                    pickle.dump(y_pred,open(predspath,'wb'))
        ROC_AUC_SCORE.append(score)
        if saving:
            print('saving report to... {}'.format(reportpath))
            pickle.dump(ROC_AUC_SCORE,open(reportpath,'wb'))
        print('======= current {} / {}'.format(epoch,total_epoch))
        print('previous best roc is {}'.format(best_score))
        print('current roc is {}'.format(score))
        print('current loss is {}'.format(logloss))
        try:
            best_round = ROC_AUC_SCORE.index(best_score)
        except ValueError:
            best_round = -1
        if len(ROC_AUC_SCORE) > patience + best_round:
            print('reach patience! end loop')
            break
            
def train_5_folds(**args):
    
    filepath = args['filepath']
    reportpath = args['reportpath']
    predspath = args['predspath']
    pred_test_file = args['predspath_test']
    batch_size = args.get('batch_size',512)
    total_epoch = args.get('total_epoch',400)
    patience = args.get('patience',5)
    saving = args.get('saving',True)
    
    train_fold_index = pickle.load(open(ENV.train_fold_index.value,'rb'))
    val_fold_index = pickle.load(open(ENV.val_fold_index.value,'rb'))

    for fold in range(0,len(train_fold_index)):
        print('!!!!!!!! Begin fold: {}'.format(fold))
        train_index = train_fold_index[fold]
        val_index = val_fold_index[fold]
        X_tra_nu = x_num_train[train_index]
        X_tra_cat = x_cal_train[train_index]
        X_val_nu = x_num_train[val_index]
        X_val_cat = x_cal_train[val_index]
        y_tra = y_train[train_index]
        y_val = y_train[val_index]
        
        x_tra_oh = x_onehot_train[train_index]
        x_val_oh = x_onehot_train[val_index]
        print('preparing train/val done!')
        print('before evaluating: {}'.format(model_file))
        model_file_evl = filepath.format(fold)
        report_file_evl = reportpath.format(fold)
        pred_file_evl = predspath.format(fold)
        pred_test_file_evl = pred_test_file.format(fold)
        model = get_NN_model()
        train_each_fold(model,
                main_n_train=X_tra_nu,
                main_n_val=X_val_nu,
                main_c_train=X_tra_cat,
                main_c_val=X_val_cat,
                y=y_tra,
                y_val=y_val,
                filepath=model_file_evl,
                reportpath=report_file_evl,
                predspath=pred_file_evl,
                batch_size=batch_size,
                total_epoch=total_epoch,
                patience=patience,
                saving=saving,
                main_oh_train=x_tra_oh,
                main_oh_val=x_val_oh)
        gc.collect()
        #### predict test
        model = load_model(model,model_file_evl)
        test_preds = model.predict({'main_numerical':x_num_test,
                                'main_categorical':x_cal_test,
                                    'main_onehot':x_onehot_test},batch_size=1500,verbose=1)
        pickle.dump(test_preds,open(pred_test_file_evl,'wb'))
        print('\n')


In [119]:
model = get_NN_model()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_numerical (InputLayer)     (None, 725)          0                                            
__________________________________________________________________________________________________
main_categorical (InputLayer)   (None, 37)           0                                            
__________________________________________________________________________________________________
batch_normalization_52 (BatchNo (None, 725)          2900        main_numerical[0][0]             
__________________________________________________________________________________________________
embedding_52 (Embedding)        (None, 37, 300)      138900      main_categorical[0][0]           
__________________________________________________________________________________________________
main_oneho

In [122]:
model_file = ENV.main_plain_nn.value
report_file = ENV.main_plain_nn_report.value
pred_file = ENV.main_plain_nn_preds.value
pred_test_file = ENV.main_plain_nn_preds_test.value

train_5_folds(filepath=model_file,
              reportpath=report_file,
              predspath=pred_file,
              predspath_test=pred_test_file,
              batch_size=512,
              total_epoch=100,
              patience=20,
              saving=True)

!!!!!!!! Begin fold: 0
preparing train/val done!
before evaluating: ../LIB/../../data/rnn/main_plain/fold_{}.hdf5
Epoch 1/1
61503/61503 [==============================] - 1s 19us/step
saving preds...
saving report to... ../LIB/../../data/rnn/main_plain/report_fold_0.pkl
======= current 0 / 100
previous best roc is 0
current roc is 0.7782676247195137
current loss is 0.2392155477475563
Epoch 1/1
61503/61503 [==============================] - 0s 7us/step
saving model to... ../LIB/../../data/rnn/main_plain/fold_0.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/main_plain/report_fold_0.pkl
======= current 1 / 100
previous best roc is 0.7782676247195137
current roc is 0.7824369623416795
current loss is 0.23842548762824753
Epoch 1/1
61503/61503 [==============================] - 0s 7us/step
saving report to... ../LIB/../../data/rnn/main_plain/report_fold_0.pkl
======= current 2 / 100
previous best roc is 0.7824369623416795
current roc is 0.7812728782761648
current loss is 0.238

Epoch 1/1
61503/61503 [==============================] - 0s 7us/step
saving report to... ../LIB/../../data/rnn/main_plain/report_fold_0.pkl
======= current 22 / 100
previous best roc is 0.7850386762486299
current roc is 0.7662898527124664
current loss is 0.24762594171224
Epoch 1/1
61503/61503 [==============================] - 0s 7us/step
saving report to... ../LIB/../../data/rnn/main_plain/report_fold_0.pkl
======= current 23 / 100
previous best roc is 0.7850386762486299
current roc is 0.765704701838153
current loss is 0.24902375906708435
Epoch 1/1
61503/61503 [==============================] - 0s 7us/step
saving report to... ../LIB/../../data/rnn/main_plain/report_fold_0.pkl
======= current 24 / 100
previous best roc is 0.7850386762486299
current roc is 0.7636896721539619
current loss is 0.25381121342676877
reach patience! end loop
48744/48744 [==============================] - 0s 9us/step


!!!!!!!! Begin fold: 1
preparing train/val done!
before evaluating: ../LIB/../../data/rnn/mai

61502/61502 [==============================] - 0s 7us/step
saving report to... ../LIB/../../data/rnn/main_plain/report_fold_2.pkl
======= current 15 / 100
previous best roc is 0.779420891485781
current roc is 0.7741932905668583
current loss is 0.24655427075323166
Epoch 1/1
61502/61502 [==============================] - 0s 7us/step
saving report to... ../LIB/../../data/rnn/main_plain/report_fold_2.pkl
======= current 16 / 100
previous best roc is 0.779420891485781
current roc is 0.7720730730502331
current loss is 0.24547090909729372
Epoch 1/1
61502/61502 [==============================] - 0s 7us/step
saving report to... ../LIB/../../data/rnn/main_plain/report_fold_2.pkl
======= current 17 / 100
previous best roc is 0.779420891485781
current roc is 0.7708261479530915
current loss is 0.24655916999396474
Epoch 1/1
61502/61502 [==============================] - 0s 7us/step
saving report to... ../LIB/../../data/rnn/main_plain/report_fold_2.pkl
======= current 18 / 100
previous best roc is 0.

Epoch 1/1
61502/61502 [==============================] - 0s 7us/step
saving report to... ../LIB/../../data/rnn/main_plain/report_fold_3.pkl
======= current 11 / 100
previous best roc is 0.7849554255447508
current roc is 0.7802441103524491
current loss is 0.2394575118015168
Epoch 1/1
61502/61502 [==============================] - 0s 7us/step
saving report to... ../LIB/../../data/rnn/main_plain/report_fold_3.pkl
======= current 12 / 100
previous best roc is 0.7849554255447508
current roc is 0.7794878207391387
current loss is 0.2388939622141673
Epoch 1/1
61502/61502 [==============================] - 0s 7us/step
saving report to... ../LIB/../../data/rnn/main_plain/report_fold_3.pkl
======= current 13 / 100
previous best roc is 0.7849554255447508
current roc is 0.779000309763535
current loss is 0.23952696608159232
Epoch 1/1
61502/61502 [==============================] - 0s 7us/step
saving report to... ../LIB/../../data/rnn/main_plain/report_fold_3.pkl
======= current 14 / 100
previous best

Epoch 1/1
61502/61502 [==============================] - 0s 7us/step
saving report to... ../LIB/../../data/rnn/main_plain/report_fold_4.pkl
======= current 6 / 100
previous best roc is 0.7887658736196304
current roc is 0.787725483793872
current loss is 0.23584412570909208
Epoch 1/1
61502/61502 [==============================] - 0s 7us/step
saving report to... ../LIB/../../data/rnn/main_plain/report_fold_4.pkl
======= current 7 / 100
previous best roc is 0.7887658736196304
current roc is 0.7864653485900261
current loss is 0.23551821507267995
Epoch 1/1
61502/61502 [==============================] - 0s 7us/step
saving report to... ../LIB/../../data/rnn/main_plain/report_fold_4.pkl
======= current 8 / 100
previous best roc is 0.7887658736196304
current roc is 0.787509828501651
current loss is 0.23523287235670665
Epoch 1/1
61502/61502 [==============================] - 0s 7us/step
saving report to... ../LIB/../../data/rnn/main_plain/report_fold_4.pkl
======= current 9 / 100
previous best ro

In [ ]:
# def get_NN_model():
#     main_numeric_input = Input(shape=(dim_main_numeric, ),name='main_numerical')
#     main_cal_sequence = Input(shape=(dim_main_cal, ),name = 'main_categorical')
#     x_main_cat = Embedding(num_words_main, main_ebd_size, trainable = True)(main_cal_sequence)
#     x_main_cat = SpatialDropout1D(0.5)(x_main_cat)
#     x_main_cat = Flatten()(x_main_cat)
#     x_main_nu = Dense(192,activation='relu')(main_numeric_input)
#     x_main_nu = Dropout(0.5)(x_main_nu)
#     x = concatenate([x_main_nu, x_main_cat])
#     x = Dense(64, activation='relu')(x)
#     x = Dropout(0.5)(x)
#     preds = Dense(1, activation="sigmoid")(x)
#     model = Model([main_numeric_input,main_cal_sequence], preds)
#     model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])
    
#     return model

In [ ]:
# def get_NN_model():
#     main_numeric_input = Input(shape=(dim_main_numeric, ),name='main_numerical')
#     main_cal_sequence = Input(shape=(dim_main_cal, ),name = 'main_categorical')
#     x_main_cat = Embedding(num_words_main, main_ebd_size, trainable = True)(main_cal_sequence)
#     x_main_cat = SpatialDropout1D(0.5)(x_main_cat)
#     x_main_cat = Bidirectional(CuDNNGRU(8, return_sequences=True))(x_main_cat)
# #     x_main_cat = Flatten()(x_main_cat)
#     x_main_cat = Conv1D(32, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x_main_cat)
#     x_main_cat = BatchNormalization()(x_main_cat)
#     avg_pool = GlobalAveragePooling1D()(x_main_cat)
#     max_pool = GlobalMaxPooling1D()(x_main_cat)
#     x_main_cat = concatenate([avg_pool, max_pool]) 

#     x_main_nu = Dense(192,activation='relu')(main_numeric_input)
#     x_main_nu = Dropout(0.5)(x_main_nu)
#     x = concatenate([x_main_nu, x_main_cat])
#     x = Dense(64, activation='relu')(x)
#     x = Dropout(0.5)(x)
#     preds = Dense(1, activation="sigmoid")(x)
#     model = Model([main_numeric_input,main_cal_sequence], preds)
#     model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])
    
#     return model

In [ ]:
# def get_NN_model():
#     main_numeric_input = Input(shape=(dim_main_numeric, ),name='main_numerical')
#     main_cal_sequence = Input(shape=(dim_main_cal, ),name = 'main_categorical')
#     x_main_cat = Embedding(num_words_main, main_ebd_size, trainable = True)(main_cal_sequence)
#     x_main_cat = SpatialDropout1D(0.5)(x_main_cat)
#     x_main_cat = Flatten()(x_main_cat)
#     x_main_nu = BatchNormalization()(main_numeric_input)
#     x_main_nu = Dense(192,activation='relu')(x_main_nu)
#     x_main_nu = Dropout(0.5)(x_main_nu)
#     x = concatenate([x_main_nu, x_main_cat])
#     x = Dense(64, activation='relu')(x)
#     x = Dropout(0.5)(x)
#     preds = Dense(1, activation="sigmoid")(x)
#     model = Model([main_numeric_input,main_cal_sequence], preds)
#     model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])
    
#     return model

# class_ratio = 1
# def train_each_epoch(model,**args):
#     #######################################
#     x_num = args['main_numerical']
#     x_cat = args['main_categorical']
#     y = args['y']
#     batch_size = args['batch_size']
#     ########################################
#     x_num,x_cat,y = shuffle(x_num,x_cat,y)
#     model.fit({'main_numerical': x_num, 'main_categorical': x_cat}, 
#               y, 
#               batch_size=batch_size, 
#               epochs=1,
#               verbose=1,
#               class_weight={0:1,1:class_ratio})
#     return model

# def load_model(model,filepath):
#     model.load_weights(filepath)
#     return model

# def save_model(model, filepath):
#     model.save_weights(filepath)
    
# def train_each_fold(model,**args):
    
#     x_num = args['main_n_train']
#     x_num_val = args['main_n_val']
#     x_cat = args['main_c_train']
#     x_cat_val = args['main_c_val']
#     y = args['y']
#     y_val = args['y_val']
#     filepath = args['filepath']
#     reportpath = args['reportpath']
#     predspath = args['predspath']
#     batch_size = args.get('batch_size',512)
#     total_epoch = args.get('total_epoch',400)
#     patience = args.get('patience',5)
#     saving = args.get('saving',True)
    
    
#     ROC_AUC_SCORE = []
#     for epoch in range(total_epoch):  
#         model = train_each_epoch(model,
#                                  main_numerical=x_num,
#                                  main_categorical=x_cat,
#                                  y=y,
#                                  batch_size=batch_size)
#         y_pred = model.predict({'main_numerical':x_num_val,
#                                 'main_categorical':x_cat_val},
#                                batch_size=3000,
#                                verbose=1)
#         score = roc_auc_score(y_val,y_pred)
#         logloss = log_loss(y_val,y_pred)
#         if len(ROC_AUC_SCORE) == 0:
#             if saving:
#                 save_model(model,filepath)
#             best_score = 0 
#             if saving:
#                 print('saving preds...')
#                 pickle.dump(y_pred,open(predspath,'wb'))
#         else:
#             best_score = max(ROC_AUC_SCORE)
#             if score >= best_score:
#                 if saving:
#                     print('saving model to... {}'.format(filepath))
#                     save_model(model,filepath)
#                     print('saving preds...')
#                     pickle.dump(y_pred,open(predspath,'wb'))
#         ROC_AUC_SCORE.append(score)
#         if saving:
#             print('saving report to... {}'.format(reportpath))
#             pickle.dump(ROC_AUC_SCORE,open(reportpath,'wb'))
#         print('======= current {} / {}'.format(epoch,total_epoch))
#         print('previous best roc is {}'.format(best_score))
#         print('current roc is {}'.format(score))
#         print('current loss is {}'.format(logloss))
#         try:
#             best_round = ROC_AUC_SCORE.index(best_score)
#         except ValueError:
#             best_round = -1
#         if len(ROC_AUC_SCORE) > patience + best_round:
#             print('reach patience! end loop')
#             break
            
# def train_5_folds(**args):
    
#     filepath = args['filepath']
#     reportpath = args['reportpath']
#     predspath = args['predspath']
#     pred_test_file = args['predspath_test']
#     batch_size = args.get('batch_size',512)
#     total_epoch = args.get('total_epoch',400)
#     patience = args.get('patience',5)
#     saving = args.get('saving',True)
    
#     train_fold_index = pickle.load(open(ENV.train_fold_index.value,'rb'))
#     val_fold_index = pickle.load(open(ENV.val_fold_index.value,'rb'))

#     for fold in range(0,len(train_fold_index)):
#         print('!!!!!!!! Begin fold: {}'.format(fold))
#         train_index = train_fold_index[fold]
#         val_index = val_fold_index[fold]
#         X_tra_nu = x_num_train[train_index]
#         X_tra_cat = x_cal_train[train_index]
#         X_val_nu = x_num_train[val_index]
#         X_val_cat = x_cal_train[val_index]
#         y_tra = y_train[train_index]
#         y_val = y_train[val_index]
#         print('preparing train/val done!')
#         print('before evaluating: {}'.format(model_file))
#         model_file_evl = filepath.format(fold)
#         report_file_evl = reportpath.format(fold)
#         pred_file_evl = predspath.format(fold)
#         pred_test_file_evl = pred_test_file.format(fold)
#         model = get_NN_model()
#         train_each_fold(model,
#                 main_n_train=X_tra_nu,
#                 main_n_val=X_val_nu,
#                 main_c_train=X_tra_cat,
#                 main_c_val=X_val_cat,
#                 y=y_tra,
#                 y_val=y_val,
#                 filepath=model_file_evl,
#                 reportpath=report_file_evl,
#                 predspath=pred_file_evl,
#                 batch_size=batch_size,
#                 total_epoch=total_epoch,
#                 patience=patience,
#                 saving=saving)
#         gc.collect()
#         #### predict test
#         model = load_model(model,model_file_evl)
#         test_preds = model.predict({'main_numerical':x_num_test,
#                                 'main_categorical':x_cal_test},batch_size=1500,verbose=1)
#         pickle.dump(test_preds,open(pred_test_file_evl,'wb'))
#         print('\n')
